In [ ]:
# day33_Application Programming Interfaces_240812
# pythonanaywhere
# https://www.pythonanywhere.com/
# Schedulted tasks
# ISS(International Space Station)

In [ ]:
# Application Programming Interface
# An API is a set of commands, functions, protocols, and objects
# that programmers can use to create software or interact with an external system.
# It is like a barrier between your program and external system.
# you're trying to use the rules that the API has prescribed to make a request to the external system for some piece of data.
# API will respond to you appropriately and give you the data that you want
# external system is like a restaurant
# API is a menu. Request and Response

In [ ]:
# API Endpoint is important
# API Endpoint is usually URL ex) api.coinbase.com
# API Request is similar to get to the bank and take the money out
# API is like a bank teller
# JSON is a flat pack.
# http://open-notify.org/Open-Notify-API/ISS-Location-Now/

In [ ]:
import requests
# the most popular module to work with API

response = requests.get(url="http://api.open-notify.org/iss-now.json")
# print(response)  # <Response [200]>
# response code 404 means that the thing that you are looking for doesn't exist.
# 1XX: Hold On, something's happening
# 2XX: Here you go. everything was successful
# 3XX: Go Away. you don't have permission to get this thing
# 4XX: You screwed up(망치다). doesn't exist
# 5XX: Server screwed up
# HTTP Status Codes: https://www.webfx.com/web-development/glossary/http-status-codes/

print(response.status_code)  # 200

if response.status_code != 200:
    # print("Error")
    raise Exception("Bad response from ISS API")

if response.status_code == 404:
    raise Exception("That resource does not exist.")
elif response.status_code == 401:
    raise Exception("You are not authorized to access this data.")
# above 4 lines of cods can be replaced by the function in the requests module
response.raise_for_status()

In [ ]:
import requests

response = requests.get(url="http://api.open-notify.org/iss-now.json")

data = response.json()["iss_position"]
print(data)  # {'latitude': '-50.5613', 'longitude': '-76.3565'}

data = response.json()["iss_position"]["longitude"]
print(data)  # -70.8506

In [ ]:
import requests

response = requests.get(url="http://api.open-notify.org/iss-now.json")

data = response.json()

longitude = data["iss_position"]["longitude"]
latitude = data["iss_position"]["latitude"]

iss_position = (longitude, latitude)
print(iss_position)  # ('-42.9635', '-38.5532')

In [ ]:
# Kanye Rest API
from tkinter import *
import requests


def get_quote():
    response = requests.get(url="https://api.kanye.rest/")
    response.raise_for_status()
    quote = response.json()["quote"]
    canvas.itemconfig(quote_text, text = quote)


window = Tk()
window.title("Kanye Says...")
window.config(padx=50, pady=50)

canvas = Canvas(width=300, height=414)
background_img = PhotoImage(file="background.png")
canvas.create_image(150, 207, image=background_img)
quote_text = canvas.create_text(150, 207, text="Kanye Quote Goes HERE", width=250, font=("Arial", 30, "bold"), fill="white")
canvas.grid(row=0, column=0)

kanye_img = PhotoImage(file="kanye.png")
kanye_button = Button(image=kanye_img, highlightthickness=0, command=get_quote)
kanye_button.grid(row=1, column=0)


window.mainloop()

In [ ]:
# API parameters: a way that allows you to give an input when tou are making yout API request
# ex) What time do you close on {Monday/Tuesday/Friday...}?
# some API allow to get parameters
# optional parameters always have default value

In [ ]:
import requests
from datetime import datetime


MY_LAT = 36.030739
MY_LONG = 129.373428


parameters = {
    "lat": MY_LAT,
    "lng": MY_LONG,
    "formatted":0
}

response = requests.get("https://api.sunrise-sunset.org/json", params=parameters)
response.raise_for_status()
data = response.json()


sunrise = data["results"]["sunrise"].split("T")[1].split(":")[0]
sunset = data["results"]["sunset"].split("T")[1].split(":")[0]
print(sunrise)
print(sunset)

time_now = datetime.now()

print(time_now.hour)

In [ ]:
# day33_final project
import requests
from datetime import datetime
import time
from mail_sending import send_mail

MY_LAT = 36.030739
MY_LONG = 129.373428


# Your position is within +5 or -5 degrees of the ISS position.
def iss_near_me():
    response = requests.get(url="http://api.open-notify.org/iss-now.json")
    response.raise_for_status()
    data = response.json()

    iss_latitude = float(data["iss_position"]["latitude"])
    iss_longitude = float(data["iss_position"]["longitude"])

    if (MY_LAT-5 <= iss_latitude <= MY_LAT+5) and (MY_LONG-5 <= iss_longitude <= MY_LONG+5):
        return True
    else:
        return False


def is_night():
    parameters = {
        "lat": MY_LAT,
        "lng": MY_LONG,
        "formatted": 0,
    }

    response = requests.get("https://api.sunrise-sunset.org/json", params=parameters)
    response.raise_for_status()
    data = response.json()
    sunrise = int(data["results"]["sunrise"].split("T")[1].split(":")[0])
    sunset = int(data["results"]["sunset"].split("T")[1].split(":")[0])

    time_now = datetime.now().hour

    if time_now >= sunset or time_now <= sunrise:
        return True


while True:
    time.sleep(60)
    if iss_near_me() and is_night():
        send_mail("#####@gmail.com",
                  "Day33 final_project",
                  "Look up!")
